# Getting Started with cuQuantum

Welcome to cuQuantum! This notebook will guide you through your first quantum circuit using NVIDIA's GPU-accelerated quantum computing SDK.

## What you'll learn:
- Setting up cuQuantum environment
- Creating quantum states
- Applying quantum gates
- Measuring quantum circuits
- Understanding GPU acceleration benefits

## 1. Setup and Installation Check

First, let's verify that all required packages are installed.

In [ ]:
# Check imports
import sys
import numpy as np

try:
    import cupy as cp
    print(f"✅ CuPy version: {cp.__version__}")
    print(f"   GPU Device: {cp.cuda.Device()}")
except ImportError:
    print("❌ CuPy not found. Install with: pip install cupy-cuda12x")
    sys.exit(1)

try:
    from cuquantum import custatevec as cusv
    print(f"✅ cuQuantum (cuStateVec) imported successfully")
except ImportError:
    print("❌ cuQuantum not found. Install with: pip install cuquantum")
    sys.exit(1)

print("\n🎉 All dependencies are ready!")

## 2. Your First Quantum State

Let's create a simple 2-qubit system. The state vector has 2^n complex amplitudes.

In [ ]:
# Define number of qubits
n_qubits = 2
state_size = 2 ** n_qubits  # 4 states: |00⟩, |01⟩, |10⟩, |11⟩

# Create state vector on GPU
state = cp.zeros(state_size, dtype=np.complex64)
state[0] = 1.0  # Initialize to |00⟩

print(f"Quantum system: {n_qubits} qubits")
print(f"State vector size: {state_size}")
print(f"Initial state |00⟩:")
print(state.get())  # Transfer from GPU to CPU for display

## 3. Applying Quantum Gates

Now let's apply some quantum gates using cuStateVec.

In [ ]:
# Create cuStateVec handle
handle = cusv.create()

# Define Hadamard gate (creates superposition)
H = np.array([[1, 1], 
              [1, -1]], dtype=np.complex64) / np.sqrt(2)

# Apply Hadamard to qubit 0
cusv.apply_matrix(
    handle, 
    state.data.ptr,  # GPU pointer to state
    cusv.cudaDataType.CUDA_C_32F,  # Data type
    n_qubits,  # Total qubits
    [0],  # Target qubit
    H.ctypes.data,  # Gate matrix
    cusv.cudaDataType.CUDA_C_32F,
    cusv.MatrixLayout.ROW,
    0  # Adjoint flag (0 = no)
)

print("After applying H to qubit 0:")
state_cpu = state.get()
for i, amp in enumerate(state_cpu):
    binary = format(i, f'0{n_qubits}b')
    if abs(amp) > 1e-10:
        print(f"|{binary}⟩: {amp.real:.4f} + {amp.imag:.4f}i")

## 4. Creating Entanglement with CNOT

The CNOT gate creates entanglement between qubits.

In [ ]:
# Define CNOT gate (control=0, target=1)
CNOT = np.array([[1, 0, 0, 0],
                 [0, 1, 0, 0],
                 [0, 0, 0, 1],
                 [0, 0, 1, 0]], dtype=np.complex64)

# Apply CNOT
cusv.apply_matrix(
    handle,
    state.data.ptr,
    cusv.cudaDataType.CUDA_C_32F,
    n_qubits,
    [0, 1],  # Control and target qubits
    CNOT.ctypes.data,
    cusv.cudaDataType.CUDA_C_32F,
    cusv.MatrixLayout.ROW,
    0
)

print("After applying CNOT (control=0, target=1):")
state_cpu = state.get()
for i, amp in enumerate(state_cpu):
    binary = format(i, f'0{n_qubits}b')
    if abs(amp) > 1e-10:
        print(f"|{binary}⟩: {amp.real:.4f} + {amp.imag:.4f}i")

print("\n🎉 You created a Bell state! |Φ⁺⟩ = (|00⟩ + |11⟩)/√2")

## 5. Measuring the Quantum State

Let's measure the probabilities of each basis state.

In [ ]:
# Compute probabilities
probabilities = np.abs(state_cpu) ** 2

# Visualize
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 5))
labels = [format(i, f'0{n_qubits}b') for i in range(state_size)]
ax.bar(labels, probabilities)
ax.set_xlabel('Basis State')
ax.set_ylabel('Probability')
ax.set_title('Bell State Measurement Probabilities')
ax.set_ylim([0, 1])
plt.grid(axis='y', alpha=0.3)
plt.show()

print("Measurement probabilities:")
for i, prob in enumerate(probabilities):
    binary = format(i, f'0{n_qubits}b')
    print(f"P(|{binary}⟩) = {prob:.4f}")

## 6. Simulating Measurements

Let's simulate many measurements to see the statistics.

In [ ]:
# Simulate 1000 shots
num_shots = 1000
np.random.seed(42)

# Sample from probability distribution
outcomes = np.random.choice(state_size, size=num_shots, p=probabilities)

# Count results
counts = np.bincount(outcomes, minlength=state_size)

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Theoretical vs Experimental
ax1.bar(labels, probabilities, alpha=0.7, label='Theoretical')
ax1.bar(labels, counts/num_shots, alpha=0.7, label='Experimental')
ax1.set_xlabel('Basis State')
ax1.set_ylabel('Probability')
ax1.set_title(f'Theoretical vs Experimental ({num_shots} shots)')
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# Shot counts
ax2.bar(labels, counts)
ax2.set_xlabel('Basis State')
ax2.set_ylabel('Counts')
ax2.set_title('Measurement Counts')
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nResults from {num_shots} measurements:")
for i, count in enumerate(counts):
    binary = format(i, f'0{n_qubits}b')
    print(f"|{binary}⟩: {count} times ({count/num_shots*100:.1f}%)")

## 7. Cleanup

Always destroy the cuStateVec handle when done.

In [ ]:
cusv.destroy(handle)
print("✅ cuStateVec handle destroyed")

## Summary

Congratulations! You've successfully:
- ✅ Created quantum states on GPU
- ✅ Applied quantum gates (H, CNOT)
- ✅ Created entanglement (Bell state)
- ✅ Measured quantum states
- ✅ Simulated measurement statistics

### Next Steps:
1. Try different gate sequences
2. Scale up to more qubits
3. Explore other notebooks:
   - `02_quantum_algorithms.ipynb` - Grover's search, QFT
   - `03_vqe_tutorial.ipynb` - Variational quantum eigensolver
   - `04_tensor_networks.ipynb` - cuTensorNet examples

### Resources:
- [cuQuantum Documentation](https://docs.nvidia.com/cuda/cuquantum/)
- [cuQuantum GitHub](https://github.com/NVIDIA/cuQuantum)
- [Example Scripts](../)